
# DSE511 — Homework #4: Sorting, Data Structures, and Big‑O

**Instructor:** Scott Emrich  
**Course:** DSE 511 – Fall 2025  
**Starter Notebook Generated:** 2025-09-10 16:29:27

---

## How to use this notebook
- Run each cell in order. Fill in any **TODO** sections.
- Keep your code **well‑commented** and your plots clearly labeled.
- For reproducibility, set the random seed as shown below.
- When you're done, export both the `.ipynb` and an `.html` of this notebook and submit on Canvas.

> **Reminder:** When making charts, use **matplotlib**, give each chart its **own figure** (no subplots), and **do not set explicit colors** unless instructed.


## Setup

In [ ]:

# Reproducibility
import numpy as np
import random
rng = np.random.default_rng(seed=511)
random.seed(511)

# Timing and plotting
import time
import matplotlib.pyplot as plt

# Utility: timing helper
def time_function(fn, *args, repeat=5, **kwargs):
    """Return the best of `repeat` wall-clock timings for fn(*args, **kwargs)."""
    best = float('inf')
    result = None
    for _ in range(repeat):
        start = time.perf_counter()
        out = fn(*args, **kwargs)
        elapsed = time.perf_counter() - start
        if elapsed < best:
            best = elapsed
            result = out
    return best, result

print("Environment ready.")



---
## Part A — Sorting Showdown

**Goal:** Empirically compare sorting algorithms and relate runtime to big‑O.

You will:
1. Generate integer arrays of increasing size.
2. Sort using:
   - Python built‑in `sorted()` (Time to sort ~ O(n log n) average/worst on random data)
   - Your own **insertion sort** (O(n²))
   - One additional algorithm of your choice (e.g., `numpy.sort`); must add
3. Measure wall‑clock runtimes as `n` grows.
4. Plot runtime vs `n` and discuss alignment with theory.


In [ ]:

# TODO: Implement insertion sort (in-place or return a new list)
def insertion_sort(arr):
    a = list(arr)
    
    #fill in; this version will return a new list using the temp variable a
    
    return a

# Sanity check
sample = [5, 2, 4, 6, 1, 3]
assert insertion_sort(sample) == sorted(sample)
print("Insertion sort sanity check passed.")


In [ ]:

# Experiment sizes (adjust if runtime too long or too short); you can also add more data points
sizes = [5_000, 10_000, 20_000]
repeat = 3

# Cap insertion sort at manageable n
ins_cap = 20_000


In [ ]:

results = {
    'n': [],
    'sorted_builtin': [],
    'insertion_sort': [],
    'numpy_sort': []  # TODO; must include below or change to another sort
}

for n in sizes:
    data = rng.integers(low=0, high=10_000_000, size=n, dtype=np.int64)
    
    t_sorted, _ = time_function(sorted, data, repeat=repeat)
    # using library similar, could be t_numpy, _ = ... and then your would add to results below
    
    if n <= ins_cap:
        t_insert, _ = time_function(insertion_sort, data, repeat=repeat)
    else:
        t_insert = np.nan
    
    results['n'].append(n)
    results['sorted_builtin'].append(t_sorted)
    results['insertion_sort'].append(t_insert)

import pandas as pd
dfA = pd.DataFrame(results)
dfA


In [ ]:

#TODO; plot linear plot from results using the log log template to help.  Need two plots for full credit

plt.figure()
for key in ['sorted_builtin', 'numpy_sort', 'insertion_sort']:
    plt.plot(dfA['n'], dfA[key], marker='o', label=key)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('n (log)')
plt.ylabel('Runtime (s, log)')
plt.title('Sorting runtime vs n (log-log)')
plt.legend()
plt.show()



---
## Part B — Data Structures in Action: Membership Testing

**Goal:** Compare membership query performance for `list` (≈ O(n)), `set` (≈ O(1) average), and `dict` (≈ O(1) average).

You will:
1. Build collections of increasing size of unique IDs/strings.
2. Generate random queries that are both present and absent.
3. Measure membership test times (`x in structure`) for each structure type.
4. Plot how query time scales with `n`.


In [ ]:

sizes_B = [10_000, 100_000, 500_000]
queries_per_size = 1_000

def gen_ids(n):
    ids = [f"id_{i:07d}" for i in range(n)]
    rng.shuffle(ids)
    return ids

def membership_timing(ids, structure_type='list'):
    if structure_type == 'list':
        coll = list(ids)
        def contains(x): return x in coll
    elif structure_type == 'set':
        coll = set(ids)
        def contains(x): return x in coll
    elif structure_type == 'dict':
        coll = {x: True for x in ids}
        def contains(x): return x in coll
    
    present = rng.choice(ids, size=queries_per_size//2, replace=False)
    absent = [f"nope_{i:07d}" for i in range(queries_per_size - len(present))]
    queries = np.concatenate([present, absent])
    rng.shuffle(queries)
    
    start = time.perf_counter()
    for q in queries:
        _ = contains(q)
    elapsed = time.perf_counter() - start
    return elapsed

records = {'n': [], 'structure': [], 'time_s': []}

for n in sizes_B:
    ids = gen_ids(n)
    for stype in ['list', 'set', 'dict']:
        t = membership_timing(ids, structure_type=stype)
        records['n'].append(n)
        records['structure'].append(stype)
        records['time_s'].append(t)

dfB = pd.DataFrame(records)
dfB


In [ ]:

plt.figure()
for stype in ['list', 'set', 'dict']:
    sub = dfB[dfB['structure'] == stype]
    plt.plot(sub['n'], sub['time_s'], marker='o', label=stype)
plt.xlabel('n')
plt.ylabel('Total time for membership queries (s)')
plt.title('Membership testing time vs n')
plt.legend()
plt.show()



---
## Part C — Reflection

In 1–2 pages, discuss:
- How well did your empirical results align with expected big‑O?
- What practical advice would you give a teammate choosing data structures and sorting methods?
- Were there any surprises in your experiments? Explain.
- How did you ensure reproducibility?
